## Problema 2:

### Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides  apresentado no trabalho TP3

 a) Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $$\gcd$$  é   $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$ .

In [1]:
def pos_condicao(a, b, r, s, t):
    if r <= 0:
        return False, "r não é positivo"

    if r != a * s + b * t:
        return False, "r não é uma combinação linear de a e b"

    for x in range(1, r):
        # Procurar combinações lineares menores
        for s_test in range(-abs(r), abs(r)):
            for t_test in range(-abs(r), abs(r)):
                if x == a * s_test + b * t_test:
                    return False, f"Existe um valor menor ({x}) que satisfaz a condição"

    return True, "Pós-condição verificada com sucesso"

def testar_pos_condicao():
    casos_teste = [
        (48, 36, 12, -1, 2),  
        (48, 36, 12, 1, -1),  
        (17, 5, 1, -2, 7)     
    ]
    
    for a, b, r, s, t in casos_teste:
        sucesso, mensagem = pos_condicao(a, b, r, s, t)
        print(f"\nTeste com a={a}, b={b}, r={r}, s={s}, t={t}")
        print(f"Resultado: {'Sucesso' if sucesso else 'Falha'}")
        print(f"Mensagem: {mensagem}")

testar_pos_condicao()



Teste com a=48, b=36, r=12, s=-1, t=2
Resultado: Falha
Mensagem: r não é uma combinação linear de a e b

Teste com a=48, b=36, r=12, s=1, t=-1
Resultado: Sucesso
Mensagem: Pós-condição verificada com sucesso

Teste com a=17, b=5, r=1, s=-2, t=7
Resultado: Sucesso
Mensagem: Pós-condição verificada com sucesso


 b) Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .

In [2]:
def lpa(a,b):
    estados = []

    r, r_linha = a,b
    s, s_linha = 1, 0
    t, t_linha = 0, 1
    q = 0 

    estados.append((r, r_linha, s, s_linha, t, t_linha, q))

    while r_linha != 0:
        q = r // r_linha

        r, r_linha, s, s_linha, t, t_linha = (
            r_linha,
            r - q * r_linha,
            s_linha,
            s - q * s_linha,
            t_linha,
            t - q * t_linha  
        )

        estados.append((r, r_linha, s, s_linha, t, t_linha, q))

    return estados

c) Construa codificações do programa LPA através de transformadores de predicados “strongest post-condition” e  prove a correção  do programa LPA.

In [3]:
def verifica_correcao(estados):
    solver = Solver()
    for i in range(len(estados) - 1):
        atual = estados[i]
        proximo = estados[i+1]
        invariante = And(
            atual.r == estados[0].r * atual.s + estados[0].r_linha * atual.t,
            atual.r_linha == estados[0].r * atual.s_linha + estados[0].r_linha * atual.t_linha,
            atual.r > 0
        )
        solver.add(invariante)
    return solver.check() == sat